In [23]:
import datetime
import os
from requests_cache import CachedSession
from tqdm.notebook import tqdm
import pandas as pd
import json

session = CachedSession(
    expire_after=datetime.timedelta(days=1), allowable_methods=["GET", "POST"]
)

key = os.getenv("CTS_V2_API_KEY")

In [24]:
from typing import Callable


OPEN_TRIALS = [
    "Active",
    "Approved",
    "Enrolling by Invitation",
    "In Review",
    "Temporarily Closed to Accrual",
    "Temporarily Closed to Accrual and Intervention",
]
OPEN_SITES = [
    "active",
    "approved",
    "enrolling_by_invitation",
    "in_review",
    "temporarily_closed_to_accrual",
]
TODAY = datetime.date.today().strftime("%Y%m%d")

def get_ctsapi_trials(start: int, **others):
    data = {
        "from": start,
        **others,
    }
    res = session.post(
        "https://clinicaltrialsapi.cancer.gov/api/v2/trials",
        json=data,
        headers={"X-API-KEY": key},
    )
    res.raise_for_status()
    return res.json()


def gather_trials(**kwargs):
    page = get_ctsapi_trials(start=0, **kwargs)
    total = page["total"]
    trials = page["data"]
    if DEBUG:
        return trials
    pbar = tqdm(total=total)
    pbar.update(len(trials))
    while len(trials) < total:
        next_page = get_ctsapi_trials(start=len(trials), **kwargs)
        trials.extend(next_page["data"])
        pbar.update(len(next_page["data"]))
    pbar.refresh()
    return trials


def get_nih_cc_open_trials_v1():
    trials = gather_trials(
        **{
            "size": 50,
            "include": ["nci_id", "nct_id", "brief_title", "current_trial_status"],
            "sites.org_postal_code": "20892",
            "sites.recruitment_status": OPEN_SITES,
            "current_trial_status": OPEN_TRIALS,
        }
    )
    df = pd.DataFrame(
        trials, columns=["nct_id", "nci_id", "current_trial_status", "brief_title"]
    )
    df.to_csv(f"nih_onsite_trials_{TODAY}.csv", index=False, encoding="utf_8_sig")


def get_nih_cc_trials_v2(
    preprocess: list[Callable[[dict,], None]],
    postprocess: list[Callable[[dict,], None]],
    writer: pd.ExcelWriter
):
    trials = gather_trials(
        **{
            "size": 50,
            "include": [
                "nci_id",
                "nct_id",
                "sites.recruitment_status",
                "sites.recruitment_status_date",
                "sites.org_name",
                "sites.org_postal_code",
            ],
            "sites.org_postal_code": "20892",
        }
    )
    for hook in preprocess:
        for trial in trials:
            hook(trial)
    df = pd.DataFrame(trials)
    for hook in postprocess:
        df = hook(df)
    df.to_excel(
        writer,
        sheet_name=f"nih_cc_trials_{TODAY}",
        index=False
    )


def get_all_trials(
    preprocess: list[Callable[[dict,], None]],
    postprocess: list[Callable[[pd.DataFrame,], pd.DataFrame]],
    sideeffect: list[Callable[[pd.DataFrame, pd.ExcelWriter], None]],
    writer: pd.ExcelWriter
):
    trials = gather_trials(
        **{
            "size": 1 if DEBUG else 50,
            "include": [
                "active_sites_count",
                "amendment_date",
                "anatomic_sites",
                "arms.type",
                # biomarkers (only the below nested fields and only if biomarkers.inclusion_indicator=TRIAL),
                "biomarkers.name",
                "biomarkers.eligibility_criterion",
                "biomarkers.nci_thesaurus_concept_id",
                "biomarkers.inclusion_indicator",
                "brief_summary",
                "brief_title",
                "ccr_id",
                "central_contact.email",
                "central_contact.name",
                "central_contact.phone",
                "central_contact.type",
                "classification_code",
                "collaborators.functional_role",
                "collaborators.name",
                "completion_date",
                "completion_date_type_code",
                "current_trial_status",
                "current_trial_status_date",
                # diseases (only the below nested fields and only if diseases.inclusion_indicator=TRIAL),
                "diseases.is_lead_disease",
                "diseases.name",
                "diseases.nci_thesaurus_concept_id",
                "diseases.inclusion_indicator",
                "eligibility.structured.accepts_healthy_volunteers",
                "eligibility.structured.gender",  # (hotfix deploying in mid/late-April that will change this to 'eligibility.structured.sex'),
                "eligibility.structured.max_age_in_years",
                "eligibility.structured.min_age_in_years",
                "keywords",
                "lead_org",
                "lead_org_cancer_center",
                "minimum_target_accrual_number",
                "nci_funded",
                "nci_id",
                "nct_id",
                "number_of_arms",
                "official_title",
                "phase",
                "primary_purpose",
                "principal_investigator",
                # prior therapy (only the below nested fields and only if prior_therapy.inclusion_indicator=TRIAL)
                "prior_therapy.eligibility_criterion",
                "prior_therapy.name",
                "prior_therapy.nci_thesaurus_concept_id",
                "prior_therapy.inclusion_indicator",
                "record_verification_date",
                "start_date",
                "start_date_type_code",
                "status_history",
                "study_model_code",
                "study_model_other_text",
                "study_population_description",
                "study_protocol_type",
                "study_source",
                "why_study_stopped",
            ],
        }
    )
    for hook in preprocess:
        for trial in trials:
            hook(trial)
    df = pd.DataFrame(trials)
    for hook in postprocess:
        df = hook(df)
    for affect in sideeffect:
        affect(df, writer)
    df.to_excel(
        writer,
        sheet_name=f"all_trials_{TODAY}",
        index=False
    )

In [25]:
NESTED_FIELDS_W_TRIAL_LEVEL = ["biomarkers", "diseases", "prior_therapy"]
NESTED_FIELDS_TO_EXPLODE = [
    "biomarkers",
    "diseases",
    "prior_therapy",
]
STATUS_MAPPING = {
    "Approved": "Not yet recruiting",
    "In Review": "Not yet recruiting",
    "Active": "Recruiting",
    "Temporarily Closed to Accrual": "Suspended",
    "Temporarily Closed to Accrual and Intervention": "Suspended",
    "Closed to Accrual": "Active, not recruiting",
    "Closed to Accrual and Intervention": "Active, not recruiting",
    "Complete": "Completed",
    "Administratively Complete": "Terminated",
    "Withdrawn": "Withdrawn",
    "Enrolling by Invitation": "Enrolling by invitation",
}


def drop_non_trial_level(trial):
    for field in NESTED_FIELDS_W_TRIAL_LEVEL:
        if field in trial:
            trial[field] = [
                item for item in trial[field] if item["inclusion_indicator"] == "TRIAL"
            ]


def cleanup_inclusion_indicators(trial):
    for field in NESTED_FIELDS_W_TRIAL_LEVEL:
        if field in trial:
            for item in trial[field]:
                del item["inclusion_indicator"]


found_one = False


def check_eligibility(trial):
    global found_one
    if found_one:
        return
    if "eligibility" in trial and "structured" in trial["eligibility"]:
        for item in trial["eligibility"]["structured"]:
            if "max_age_in_years" in item or "gender" in item:
                print("Gender or max_age_in_years exists")
                found_one = True
                break


def count_n_nested_fields(trial):
    for field in NESTED_FIELDS_TO_EXPLODE:
        if field in trial:
            if isinstance(trial[field], list):
                print(len(trial[field]))


field_length_max = {}
max_field_trial_id = {}
max_field_contents = {}


def count_size_of_fields(trial):
    for field in trial:
        length = 0
        contents: str
        if isinstance(trial[field], str):
            contents = trial[field]
            length = len(contents)
        elif isinstance(trial[field], (list, dict)):
            contents = json.dumps(trial[field])
            length = len(contents)
        else:
            continue
        if field in field_length_max:
            if length > field_length_max[field]:
                field_length_max[field] = length
                max_field_trial_id[field] = trial["nct_id"]
                max_field_contents[field] = contents
        else:
            field_length_max[field] = length
            max_field_trial_id[field] = trial["nct_id"]
            max_field_contents[field] = contents


def explode_nested(df):
    for field in NESTED_FIELDS_TO_EXPLODE:
        if field in df.columns:  # Check if the column exists before exploding
            df = df.explode(field)
        print(df.shape)
    return df


def map_ctrp_stat_to_ctg(df: pd.DataFrame) -> pd.DataFrame:
    df["current_trial_status (mapped)"] = df["current_trial_status"].apply(
        lambda stat: STATUS_MAPPING[stat]
    )
    df = df.drop(["current_trial_status"], axis=1)
    return df


non_matching_postal_codes = set()


def remove_non_nih_cc_sites(trial):
    sites_filtered = []
    for site in trial["sites"]:
        if not site["org_postal_code"].startswith("20892"):
            non_matching_postal_codes.add(site["org_postal_code"])
        else:
            if sites_filtered:
                found = False
                for site_filtered in sites_filtered:
                    if site == site_filtered:
                        found = True
                        break
                if not found:
                    sites_filtered.append(site)
            else:
                sites_filtered.append(site)
    if len(sites_filtered) > 1:
        print(trial["nct_id"])
    trial["sites"] = sites_filtered


def set_column_names(df: pd.DataFrame) -> pd.DataFrame:
    columns = [
        "nci_id",
        "nct_id",
        "ccr_id",
        "lead_org",
        "lead_org_cancer_center",
        "principal_investigator",
        "collaborators",
        "phase",
        "official_title",
        "brief_title",
        "brief_summary",
        "study_protocol_type",
        "study_source",
        "primary_purpose",
        "nci_funded",
        "keywords",
        "eligibility",
        "study_population_description",
        "study_model_code",
        "arms",
        "number_of_arms",
        "anatomic_sites",
        "biomarkers",
        "diseases",
        "study_model_other_text",
        "prior_therapy",
        "start_date",
        "start_date_type_code",
        "completion_date",
        "completion_date_type_code",
        "current_trial_status (mapped)",
        "current_trial_status_date",
        "why_study_stopped",
        "status_history",
        "record_verification_date",
        "amendment_date",
        "minimum_target_accrual_number",
        "central_contact",
        "classification_code",
        "active_sites_count",
    ]
    return df[columns]


def set_column_names_cc(df: pd.DataFrame) -> pd.DataFrame:
    columns = ["nct_id", "sites", "nci_id"]
    return df[columns]


def write_earliest_status(df: pd.DataFrame, writer: pd.ExcelWriter) -> None:
    df_stat_hist = df.loc[:, ["nci_id", "status_history"]]

    def get_prop(el: dict, prop: str):
        return el[prop]

    df_stat_hist_exp = df_stat_hist.explode("status_history")
    df_stat_hist_exp["status"] = df_stat_hist_exp["status_history"].apply(
        get_prop, args=("status",)
    )
    df_stat_hist_exp["status_date"] = df_stat_hist_exp["status_history"].apply(
        get_prop, args=("status_date",)
    )
    df_stat_hist_exp = df_stat_hist_exp.drop("status_history", axis=1)
    expected_len = df_stat_hist["status_history"].transform(len)
    assert expected_len.sum() == len(df_stat_hist_exp)
    df_earliest_stat = (
        df_stat_hist_exp.sort_values(by="status_date")
        .groupby(["nci_id", "status"])
        .nth(0)
    )
    df_earliest_stat = df_earliest_stat.reset_index(drop=True)
    df_earliest_stat.sort_values(["nci_id", "status_date"]).to_excel(
        writer, sheet_name=f"earliest_status_{TODAY}", index=False
    )


DEBUG = False
with pd.ExcelWriter(
    f"clinical_trials_tracker_{TODAY}.xlsx",
    engine="xlsxwriter",
) as writer:
    get_nih_cc_trials_v2(
        preprocess=[remove_non_nih_cc_sites],
        postprocess=[set_column_names_cc],
        writer=writer,
    )
    get_all_trials(
        preprocess=[
            drop_non_trial_level,
            cleanup_inclusion_indicators,
            check_eligibility,
            count_size_of_fields,
        ],
        postprocess=[
            map_ctrp_stat_to_ctg,
            set_column_names,
        ],
        sideeffect=[write_earliest_status],
        writer=writer,
    )

  0%|          | 0/756 [00:00<?, ?it/s]

NCT03816345
NCT05327010
NCT05685602
NCT05836571
NCT04704661
NCT05687110
NCT06311227
NCT04981509
NCT05333458
NCT05691504
NCT05687136
NCT04550494
NCT02203526
NCT04840589
NCT01419561
NCT05715281
NCT01174121
NCT03388632
NCT03366116
NCT05142241
NCT02423057
NCT01688999
NCT00321555
NCT02015104
NCT01621568
NCT04458922
NCT03872427
NCT02275533
NCT01851369
NCT01748825
NCT01572493
NCT01051635
NCT03502733
NCT02723864
NCT04491942
NCT03218826
NCT01534598
NCT01222754
NCT02298959
NCT04535401
NCT02407405
NCT01572480
NCT00942877
NCT00924027
NCT00923013
NCT00092222
NCT01391962
NCT04595747
NCT02345265
NCT03141684
NCT01362803
NCT01273168
NCT04616534
NCT04294628
NCT02496208
NCT02326844
NCT01362790
NCT02362451
NCT02315625
NCT02203513
NCT01838642
NCT01306045
NCT02111863
NCT01814046
NCT02062359
NCT01583686
NCT01218867
NCT02153905
NCT02192541
NCT01875601
NCT01352962
NCT00926640
NCT01239368
NCT01273155
NCT01829711
NCT00924170
NCT00923845
NCT02015065
NCT01867333
NCT01130519
NCT01030900
NCT02019693
NCT01875250
NCT0

  0%|          | 0/24503 [00:00<?, ?it/s]

Gender or max_age_in_years exists


#### Investigation

In [12]:
highest = {pair for pair in field_length_max.items() if pair[1] > 20000}
for item in highest:
    print(item)

('diseases', 21625)


In [13]:
types = set()
lens = set()
for item in non_matching_postal_codes:
    if len(item) == 4:
        print(item)
    types.add(type(item))
    lens.add(len(item))
types, lens

6009
8011
2031
6008
4101
1145
4029
3052


({str}, {4, 5, 7, 8, 10})